# TrainModel

Trainanje Modela pomoću tflearna i tensorflowa, djelom optimizirano za rad na GPU. Sprema Snapshotove i one modele koji su imali najbolji val_acc



In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jun 29 01:06:46 2018

@author: User
"""



from __future__ import division, print_function, absolute_import
import tflearn 
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d, avg_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression

#import tensorflow as tf

from tqdm import tqdm
import os
import json
import numpy as np
from random import shuffle
import tensorflow as tf

'''
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4
config.allow_soft_placement = True
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
'''


DESTINATION ='/home/bosnmar/MachineLearning/Production/TrainingSet/FINAL_TRAINING/TESTDATA/'
LR = 0.0001
IMGSIZE1 = 60
IMGSIZE2 = 40

MODELNAME = "cnnModelName"

def loadData():
    trainingData = np.empty([0, 2])
    for data in tqdm(os.listdir(DESTINATION)):
        #trainingData.concatenate( trainingData, np.load(DESTINATION + data))
        myData = np.load(DESTINATION + data)
        trainingData = np.append(trainingData, myData, axis=0)
        #print (myData.shape)
        #print("Loaded class: {}".format(data))
        #print(trainingData.shape)

    return trainingData

def getall():
    #trainDat = np.load('trainingDataNumNoOtherClass.npz')
    #trainData = loadData()
    
    trainData = loadData()
    
    print("\nData loaded, ready to start working on it!")
    print("\nData shape: {}".format(trainData.shape))
    print("\nData len: {}".format(trainData.shape))
    
    ######## ovaj dio izmedju ovih komentara pokrenes samo jednom da ti izgenerira indekse, tako da learnas i validiras uvijek 
    #### na istim podacima

    ########
    permut = []
    with open('permut2.txt', 'r') as f:
        onlyline = f.readline()[1:-1]
        onlyline = onlyline.split(", ")
        permut = list(map(lambda x: int(x), onlyline))
    trainDat = trainData[permut[:-100000]]
    testData = trainData[permut[-100000:]]

    X = np.array([i[0] for i in trainDat]).reshape(-1, IMGSIZE1, IMGSIZE2, 1)
    Y = np.array([i[1] for i in trainDat])

    test_x = np.array([i[0] for i in testData]).reshape(-1, IMGSIZE1, IMGSIZE2, 1)
    test_y = train_Y = np.array([i[1] for i in testData])

    #print (len(X))
    return X, Y, test_x, test_y

def getLabels():
    with open('imgLabels.json') as f:
        return json.load(f)
    
    
    
if __name__=="__main__":
    labels = getLabels()
    print("Started creating new npy files (converting pictures to numpy arrays with image labels)\nData is already in np, so we skip to loading data")
    #createTrainData(imglab, labels)
    print("loading data from npy files")
    #tflearn.init_graph(num_cores=1,gpu_memory_fraction=0.5)

    network = input_data(shape=[None, IMGSIZE1, IMGSIZE2, 1], name='input')
    network = conv_2d(network, 50, 5, activation='relu' )
    network = max_pool_2d(network, 2)
    #Normalization purposed to speed up training process
    network = local_response_normalization(network)
    network = conv_2d(network, 50, 5, activation='relu')
    network = avg_pool_2d(network, 2)
    network = local_response_normalization(network)
    network = conv_2d(network, 100, 5, activation='relu')
    network = avg_pool_2d(network, 2)

    network = fully_connected(network, 1000, activation='relu')
    network = fully_connected(network, 1000, activation='relu')
    network = fully_connected(network, 200, activation='relu')
    network = fully_connected(network, 25, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=LR,
                        loss='categorical_crossentropy', name='targets')

    X, Y, test_x, test_y = getall()
    print('X shape' + str(list(np.shape(X))))
    print('test_x shape' + str(list(np.shape(test_x))))
    model = tflearn.DNN(network, tensorboard_dir='log' + MODELNAME, best_checkpoint_path='bestmodel')

    if (os.path.exists('{}.meta'.format(MODELNAME))):
        model.load(MODELNAME)
        print('model loaded!')

    print("\n\n")
    print("Starting with convnet. Fingers crossed\n")

    model.fit({'input': X}, {'targets': Y}, n_epoch=3, validation_set=({'input': test_x}, {'targets': test_y}), batch_size = 50, shuffle = True, snapshot_step=20000, show_metric=True, run_id=MODELNAME)
    
    model.save(MODELNAME)